<a href="https://colab.research.google.com/github/NoCodeProgram/deepLearning/blob/main/cnn/cnn2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt

if torch.backends.mps.is_available():
    my_device = torch.device('mps')
elif torch.cuda.is_available():
    my_device = torch.device('cuda')
else:
    my_device = torch.device('cpu')

print(my_device)

cuda


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Load CIFAR10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=4)


100%|██████████| 170498071/170498071 [00:02<00:00, 78720528.67it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified


In [3]:
# Define the simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 5) # 3 input channels, 6 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.conv3 = nn.Conv2d(32, 64, 5) # Adding the third convolutional layer

        # CIFAR10 images are 32x32. Three conv layers without pooling or stride reduce the size to 24x24, 20x20, then 16x16.
        self.fc1 = nn.Linear(64 * 20 * 20, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # 10 classes in CIFAR10

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.conv3(x)           # Adding the forward pass for the third convolutional layer
        x = torch.relu(x)
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x



# Create the model
net = SimpleCNN()


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)

In [ ]:
net.to(my_device)
num_epochs = 100
for epoch in range(num_epochs):
    net.train()
    for batch_idx, (data, label) in enumerate(trainloader):
        data, label = data.to(my_device), label.to(my_device)
        scores = net(data)
        loss = criterion(scores, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    net.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, label in testloader:
            data, label = data.to(my_device), label.to(my_device)
            scores = net(data)
            loss = criterion(scores, label)
            val_loss += loss.item() * data.size(0)

            predicted = scores.argmax(dim=1)
            correct += predicted.eq(label).sum().item()

    val_loss /= len(testloader.dataset)
    val_accuracy = 100. * correct / len(testloader.dataset)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Epoch [1/2300], Training Loss: 1.6078, Validation Loss: 1.6847, Validation Accuracy: 39.97%
Epoch [2/2300], Training Loss: 1.2575, Validation Loss: 1.5395, Validation Accuracy: 44.33%
Epoch [3/2300], Training Loss: 1.1016, Validation Loss: 1.4794, Validation Accuracy: 46.01%
Epoch [4/2300], Training Loss: 0.9029, Validation Loss: 1.4204, Validation Accuracy: 48.65%
Epoch [5/2300], Training Loss: 0.7592, Validation Loss: 1.3979, Validation Accuracy: 49.34%
Epoch [6/2300], Training Loss: 0.8330, Validation Loss: 1.3740, Validation Accuracy: 51.11%
Epoch [7/2300], Training Loss: 0.6635, Validation Loss: 1.3432, Validation Accuracy: 52.18%
Epoch [8/2300], Training Loss: 0.6160, Validation Loss: 1.3048, Validation Accuracy: 53.50%
